# TSMixer

In [ ]:
!pip install pytorch-tsmixer

In [ ]:
import torch

def smape_loss(predicted, target):
    """
    Compute Symmetric Mean Absolute Percentage Error (SMAPE) loss between predicted and target tensors.
    Args:
        predicted (torch.Tensor): Predicted values.
        target (torch.Tensor): Target values.
    Returns:
        torch.Tensor: SMAPE loss.
    """
    numerator = torch.abs(predicted - target)
    denominator = (torch.abs(predicted) + torch.abs(target)) / 2.0
    smape = torch.mean(numerator / denominator) * 100.0
    return smape

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtsmixer import TSMixer
import pandas as pd 
import sys
from torch.utils.data import DataLoader, TensorDataset
import random


def dataPreprocessor(fileName, dropScore, seq_len, pred_len, input_channels, output_channels):

    ts = pd.read_csv(fileName)
    # -------------- Data processing --------------
    cumulative_clmsn = []
    for clmn_name in ts:
        if "Cumulative" in clmn_name:
            cumulative_clmsn.append(clmn_name)
        
        if dropScore == True and "score" in clmn_name:
            cumulative_clmsn.append(clmn_name)
            
    for entry in cumulative_clmsn:
        ts.drop(columns=entry, inplace=True)
    # -------------- Data processing --------------
        
    # Drop date
    data = ts.select_dtypes(include=['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8', 'uint8'])
    data = data.fillna(0)
    

     # Model parameters
    sequence_length = seq_len    
    prediction_length = pred_len
    print(input_channels)
    print(len(data.columns) -1)
    assert(input_channels == len(data.columns) - 1)

    X_train = torch.empty(0, sequence_length, input_channels)
    y_train = torch.empty(0, prediction_length, output_channels)

    X_val = torch.empty(0, sequence_length, input_channels)
    y_val = torch.empty(0, prediction_length, output_channels)

    X_test = torch.empty(0, sequence_length, input_channels)
    y_test = torch.empty(0, prediction_length, output_channels)

    for start in range(1, len(data) - sequence_length - prediction_length + 1):
        X_inst = torch.tensor(data.iloc[start:start + sequence_length, :-1].values, dtype=torch.float32)
        y_inst = torch.tensor(data.iloc[start + sequence_length : start + sequence_length + prediction_length, -1:].values, dtype=torch.float32)    

        coin_flip = random.randint(1, 100)

        if coin_flip <= 80:
            X_train = torch.cat((X_train, X_inst.unsqueeze(0)), dim = 0)
            y_train = torch.cat((y_train, y_inst.unsqueeze(0)), dim = 0)
        elif coin_flip <= 90:
            X_val = torch.cat((X_val, X_inst.unsqueeze(0)), dim = 0)
            y_val = torch.cat((y_val, y_inst.unsqueeze(0)), dim = 0)
        else:
            X_test = torch.cat((X_test, X_inst.unsqueeze(0)), dim = 0)
            y_test = torch.cat((y_test, y_inst.unsqueeze(0)), dim = 0)

    
    return [X_train, y_train, X_val, y_val, X_test, y_test]


def train_model(model, criterion, optimizer, train_dataloader):
    model.train()
    running_loss = 0.0

    for X, y in train_dataloader:
        # Extract a single batch from the dataset                   
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X) 
        loss = criterion(outputs, y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()    
        running_loss += loss.item() * X.size(0)

    return running_loss / len(train_dataloader.dataset)
    

def evaluate_model(model, criterion, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            outputs = model(X)
            loss = criterion(outputs, y)
            running_loss += loss.item() * X.size(0)
    return running_loss / len(dataloader.dataset)

def findBestModelAndTest(X_train, y_train, X_val, y_val, X_test, y_test, seq_len, pred_len, input_channels, output_channels, batch_size, num_epochs):   
    train_dataset = TensorDataset(X_train, y_train)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    val_dataset = TensorDataset(X_val, y_val)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    test_dataset = TensorDataset(X_test, y_test)
    test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle= False)

    # Create the TSMixer model
    model = TSMixer(seq_len, pred_len, input_channels, output_channels)

    # Loss function and optimizer
    criterion = smape_loss
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    # Train the model
    best_val_loss = float('inf')
    best_model = None
    for epoch in range(num_epochs):
        train_loss = train_model(model, criterion, optimizer, train_dataloader)
        val_loss = evaluate_model(model, criterion, val_dataloader)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict()
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} (Model saved)')
        else:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        # You can add early stopping here if val_loss does not decrease for a certain number of epochs
        # You can also adjust learning rate based on validation loss

    # Load the best model
    model.load_state_dict(best_model)    

    
    test_loss = evaluate_model(model, criterion, test_dataloader)
    print(f'Best Validation Loss: {best_val_loss:.4f}, Test Loss: {test_loss:.4f}')

In [ ]:
dropScore = 0
seq_len = 21
pred_len = 4
input_channels = 12 - dropScore # HARDCODED
output_channels = 1
batch_size = 32

allData = dataPreprocessor("covidDataWithSentiment.csv", dropScore=dropScore, seq_len=seq_len, pred_len=pred_len, input_channels=input_channels, output_channels=output_channels)

findBestModelAndTest(*allData, seq_len=21, pred_len=4, input_channels=input_channels, output_channels=output_channels, batch_size=batch_size, num_epochs=1400)
